In [16]:
#Check para ver si lee mi key

import os
from dotenv import load_dotenv
import requests
import pandas as pd
from io import StringIO
import time

# 1) Cargar el archivo .env
load_dotenv()

# 2) Leer la variable del entorno
API_KEY = os.getenv("AEMET_API_KEY")

# 3) Comprobar que se ha cargado
if API_KEY is None:
    print("No se ha encontrado AEMET_API_KEY. Revisa tu archivo .env")
else:
    print("✅ API key cargada correctamente. Longitud:", len(API_KEY))
    # Opcional: mostrar solo los primeros caracteres para comprobar
    print("Inicio de la key:", API_KEY[:10], "... (oculto)")



✅ API key cargada correctamente. Longitud: 281
Inicio de la key: eyJhbGciOi ... (oculto)


In [17]:
# Estación de Peñafiel (código que ves en la web: 2166Y)
STATION_ID = "2166Y"

# Endpoint /api/ de climatologías mensuales/anuales:
BASE_URL = (
    "https://opendata.aemet.es/opendata/api/"
    "valores/climatologicos/mensualesanuales/datos/"
    "anioini/{year}/aniofin/{year}/estacion/{station}"
)

print("✅ Configuración lista para estación", STATION_ID)


✅ Configuración lista para estación 2166Y


In [18]:
# Crea función para descargar datos ajustado con control de errores

def get_aemet_year(year):
    """
    Descarga climatologías mensuales/anuales de AEMET para un año y la estación 2166Y.
    Si hay errores HTTP (500, 404, etc.) en la API o en la URL 'datos',
    devuelve un DataFrame vacío y sigue.
    """

    # 1. Construir la URL del endpoint API
    url_api = BASE_URL.format(year=year, station=STATION_ID)

    # 2. Llamar a la API con tu key
    headers = {"api_key": API_KEY}
    try:
        resp = requests.get(url_api, headers=headers, timeout=30)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(f"[{year}] ❌ Error HTTP en la llamada al endpoint API: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"[{year}] ❌ Error genérico llamando al endpoint API: {e}")
        return pd.DataFrame()

    info = resp.json()
    estado = info.get("estado")
    desc = info.get("descripcion")

    if estado != 200:
        print(f"[{year}] ❌ Error de API: estado={estado}, descripcion={desc}")
        return pd.DataFrame()

    data_url = info["datos"]  # URL sh/... fresca

    # 3. Descargar los datos desde 'datos'
    try:
        r_data = requests.get(data_url, timeout=30)
        r_data.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(f"[{year}] ❌ Error HTTP al descargar 'datos': {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"[{year}] ❌ Error genérico al descargar 'datos': {e}")
        return pd.DataFrame()

    # 4. Intentar primero JSON (para detectar 'datos expirados' o datos en JSON)
    try:
        data_json = r_data.json()

        # Caso dict con estado=404 → datos expirados
        if isinstance(data_json, dict) and data_json.get("estado") == 404:
            print(f"[{year}] ⚠️ 'datos expirados' en la URL de datos. Prueba más tarde.")
            return pd.DataFrame()

        # Caso lista de dicts → datos OK en JSON
        if isinstance(data_json, list):
            df = pd.DataFrame(data_json)
            df["year"] = year
            print(f"[{year}] ✅ OK JSON con {len(df)} filas")
            return df

    except ValueError:
        # No era JSON → probamos CSV
        pass

    # 5. Intentar CSV (lo habitual en climatologías mensuales/anuales)
    try:
        csv_buf = StringIO(r_data.text)
        df = pd.read_csv(csv_buf, sep=";")
        df["year"] = year
        print(f"[{year}] ✅ OK CSV con {len(df)} filas")
        return df
    except Exception as e:
        print(f"[{year}] ❌ Error leyendo CSV: {e}")
        return pd.DataFrame()


In [19]:
# Creamos bucle
# De 2003 a 2024 inclusive
time.sleep(3)
years = list(range(2003, 2025))  # 2003, 2004, ..., 2024

dfs = []

for y in years:
    df_y = get_aemet_year(y)
    if not df_y.empty:
        dfs.append(df_y)
    else:
        print(f"[{y}] ⚠️ Sin datos válidos, no se añade al total.")

if dfs:
    df_aemet = pd.concat(dfs, ignore_index=True, sort=False)
    print(f"\n✅ Concatenación completa: {len(df_aemet)} filas en total.")
else:
    df_aemet = pd.DataFrame()
    print("❌ No se han obtenido datos para ningún año.")

df_aemet.head()


[2003] ✅ OK JSON con 13 filas
[2004] ✅ OK JSON con 13 filas
[2005] ✅ OK JSON con 13 filas
[2006] ✅ OK JSON con 13 filas
[2007] ✅ OK JSON con 13 filas
[2008] ✅ OK JSON con 13 filas
[2009] ✅ OK JSON con 13 filas
[2010] ✅ OK JSON con 13 filas
[2011] ✅ OK JSON con 13 filas
[2012] ✅ OK JSON con 13 filas
[2013] ✅ OK JSON con 13 filas
[2014] ✅ OK JSON con 13 filas
[2015] ✅ OK JSON con 13 filas
[2016] ✅ OK JSON con 13 filas
[2017] ✅ OK JSON con 13 filas
[2018] ✅ OK JSON con 13 filas
[2019] ✅ OK JSON con 13 filas
[2020] ✅ OK JSON con 13 filas
[2021] ✅ OK JSON con 13 filas
[2022] ✅ OK JSON con 13 filas
[2023] ✅ OK JSON con 13 filas
[2024] ✅ OK JSON con 13 filas

✅ Concatenación completa: 286 filas en total.


,indicativo,fecha,p_max,np_100,np_001,np_300,p_mes,np_010,tm_min,ta_max,...,nt_00,ti_max,tm_mes,tm_max,year,e,hr,n_cub,n_des,n_nub
0,2166Y,2003-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2003,NaN,NaN,NaN,NaN,NaN
1,2166Y,2003-7,5.0(15),0,6,0,11.0,3,NaN,NaN,...,NaN,NaN,NaN,NaN,2003,NaN,NaN,NaN,NaN,NaN
2,2166Y,2003-8,8.6(27),0,10,0,27.2,6,NaN,NaN,...,NaN,NaN,NaN,NaN,2003,NaN,NaN,NaN,NaN,NaN
3,2166Y,2003-11,11.6(23),1,15,0,48.8,8,NaN,NaN,...,NaN,NaN,NaN,NaN,2003,NaN,NaN,NaN,NaN,NaN
4,2166Y,2003-5,6.8(31),0,7,0,26.2,6,NaN,NaN,...,NaN,NaN,NaN,NaN,2003,NaN,NaN,NaN,NaN,NaN


In [20]:
df_aemet.to_csv("aemet_vino_raw.csv", index=False)